## Script to select most frequent combined variables

In [ ]:
## This script is used to select combined variables for further consideration.

require(XML)
require(tidyverse)
require(stringr)

input.dir <- "./output_2"

files.v <- dir(path=input.dir, pattern=".*xml")



generate_vars <- function(var.list, df) {
  a <- unlist(var.list)
  b <- df[, a]
  if (length(a) > 1) {
    c <- apply(b, 1, paste0, collapse = "/")
  } else {
    c <- b 
  }
  return(c)
}


generate_var_names <- function(var.list) {
  a <- unlist(var.list)
  b <- paste0(a, collapse = "_&_")
  return(b)
}


#### This script generates combined variables and allows for the calculation of the frequency with which a particular variable type has values or is unused.

* **require(XML), require(tidyverse),** and **require(stringr)** load the packages necessary for the script.
* **input.dir <- "./output_2"** creates a character vector containing the directory holding the target XML files.
* **files.v <- dir(path=input.dir, pattern=".&#42;xml")** creates a character vector containing the names of the XML files in the specified directory. 
* **generate_vars <- function(var.list, df) { ... }** creates a user-defined function to be applied with **sapply( )** to generate new combined variables.
  + The function takes two arguments. The first is a list object containing nested lists of the the names of the simplex variables to be combined. This list will be generated later in this script. Second, a data frame object with a value for each simplex variable for each token in the input file. This data frame too will be generated later in the srcript.
  + **a <- unlist(var.list)** creates a character vector with the names of the simplex variable(s) from the input nested list.
  + **b <- df[, a]** creates a data frame object by selecting those columns in the input data frame with the names contained in the character vector **a**.
  + **if (length(a) > 1) { ... }** selects for instances of the character vector **a** which contain more than one variable name. Such instances must be treated differently than simplex variables.
    + **c <- apply(b, 1, paste0, collapse = "/")** creates a matrix with a row for each input row (one for each token in input file) and a column for the new combined variable.
      + **apply( )** applies the specified function to each row or column of the input. The first argument is the input object and the second argument, **1**, indicates a row-by-row treatment. The function to be applied is **paste0(x, collapse = "/")**. This function joins the input (two or more variable values) into a single character string joined by a forward slash character.
  + **else { ... }** selects intances in which **a** contains only one element. Such instances represent simplex variables.
    + **c <- b** passes the column of simplex variables to the output variable
  + **return(c)** passes the output variable to the matrix code.  
* **generate_var_names <- function(var.list) { ... }** creates a user-defined function to generate names for the new variables. The argument is the list object containing nested lists of the names of the simplex variables to be combined.
  + **a <- unlist(var.list)** creates a character vector with the names of the simplex variables.
  + **b <- paste0(a, collapse = "_&_")** combines multiple elements of vector **a** into a single character string joined by the characters "_&_". 
  + ** return(b)** passes **b** to the matrix code.
  


In [ ]:

doc.object <- xmlTreeParse(file.path(input.dir, files.v[1]), useInternalNodes=TRUE)


word.nodes <- getNodeSet(doc.object, "//word")


word.list <- xmlApply(word.nodes, xmlToList)




var_names.v <- map(word.list, names) %>%
  unlist() %>%
  unique()

var_names.v <- var_names.v[-which(var_names.v == "insertion_id" | var_names.v == "artificial") ]


var_names.v <- var_names.v[12:length(var_names.v)] # drop non-expanded attributes as variables

###########

#### The next step is to generate a list of simplex variable names to be used in making the combined variables

* **doc.object <- xmlTreeParse(file.path(input.dir, files.v[1]), useInternalNodes=TRUE)** creates an object containing a representation of the xml structure of the input file.  Here the first file in the input directory is specified (** files.v[1]**). All files in the directory should have the same set of attributes to be used for variables.
* **word.nodes <- getNodeSet(doc.object, "//word")** creates a xmlNodeSet object containing all word nodes from the input. 
* **word.list <- xmlApply(word.nodes, xmlToList)** creates an R list object containing the data from each word node. The data are more easily manipulated in this format.
* **var_names.v <-** creates a character vector containing the output of the associated code.
  + **map(word.list, names)** applies the function **names( )** to each element in **word.list**. The result is a large list object (number of list elements = words in word.list). Each list element contains a character vector of attribute names.
  + **%>% unlist()** converts the input to a large character vector.
  + **%>%  unique()** removes redundant elements and returns unique names.
* **var_names.v <- var_names.v[-which(var_names.v == "insertion_id" | var_names.v == "artificial") ]** removes attributes specific to ellipsis nodes from the vector. The values of the deleted attributes have no relevance in syntactic analysis.
* **var_names.v <- var_names.v[12:length(var_names.v)]** limits the vector contents to only those attributes that have been given easily interpretable names. The attributes eliminated are those created by the original dependency annotation. Their values have been included in the retained elements.


In [ ]:

holder.tib <- tibble(variable_name="blank")


for(i in seq_along(files.v)) {
  
  # read xml structure from file to .R object
  doc.object <- xmlTreeParse(file.path(input.dir, files.v[i]), useInternalNodes=TRUE)
  
  # extract all <word> elements and children into XmlNodeList object
  word.nodes <- getNodeSet(doc.object, "//word")
  
  
  word.list <- xmlApply(word.nodes, xmlToList)
  
 
  for (j in seq_along(var_names.v)) {
    
    if ( j == 1) {
      
      base.df <- word.list %>% map_chr(var_names.v[j]) %>%
        data.frame(check.names = FALSE, stringsAsFactors = FALSE)
      
    } else {
      
      base.df <- word.list %>% map_chr(var_names.v[j]) %>%
        cbind(base.df,  ., stringsAsFactors = FALSE)
      
    }
    
  } # end of j loop
  
  colnames(base.df) <- var_names.v
  
  
  
 
  
  
  selected_vars.list <- list() # make empty list object to store result of loop
  nomina.v <- NULL # make empty vector to store names
  
  
  for (k in 1:3) { 
    selected_vars.list[[k]] <- combn(var_names.v, k, simplify = FALSE) # make all possible combinations of variables
    nomina.v <- paste(length(var_names.v), "Choose",  k, collapse = " ") %>% # create names for elements in list
      append(nomina.v, .)
  }
  
  names(selected_vars.list) <- nomina.v  # assign names to list elements
  
 
  
  new_vars.m <- matrix(nrow = nrow(base.df), ncol = 1)
  
  
  for (m in seq_along(selected_vars.list)) {
    
 
    a <- sapply(selected_vars.list[[m]], generate_vars, df = base.df)
    
    nomina <- sapply(selected_vars.list[[m]], generate_var_names)
    
    colnames(a) <- nomina
    
    new_vars.m <- cbind(new_vars.m, a)
    
    end_time <- Sys.time()
    
   
  }
  
  new_vars.m <- new_vars.m[, -1]
  
  
  new_vars.m[str_detect(new_vars.m, "NA")] <- NA # add logical NA to cells
  
  
  new_vars.m[str_detect(new_vars.m, "NULL")] <- NA # add logical NA to cells
  
  new_vars.m <-tolower(new_vars.m)
  
  vars.tib <- as_tibble(new_vars.m)
  
  
  file_var_ct.tib <-   gather(vars.tib, variable_name, variable_value, na.rm = TRUE) %>%
    group_by(variable_name) %>%
    summarize(n())
  
 
  
  holder.tib <- bind_rows(holder.tib, file_var_ct.tib)
  
  
}






#### The next step is a set of nested loops to generate new variables and their values


* **holder.tib <- tibble(variable_name="blank")** creates a tibble object to contain the results of the loop series. The result is a tibble (a kind of data frame) with one column, called **variable_name**, with one row. The content of the single data cell is the string "blank".
* **for(i in seq_along(files.v)) { ... }** is the main loop, which iterates through the files in the **files.v** vector.
  + **doc.object <- xmlTreeParse(file.path(input.dir, files.v[i]), useInternalNodes=TRUE)** creates an R object containing a representation of the XML structure of the input file.
  + **word.nodes <- getNodeSet(doc.object, "//word")** creates an XMLNodeSet object containing all word elements of the input files.
  + **word.list <- xmlApply(word.nodes, xmlToList)** converts the NodeSet object to a regular R list object.
  + **for (j in seq_along(var_names.v)) { ... }** begins a loop to create a data frame with each word of the input file as a row and each attribute as a column. The loop iterates through the attribute/variable names in **var_names.v**.
    + **if ( j == 1) { ... }** selects for the first iteration, which must create the data frame.
      + **base.df <-** creates the data frame containing the results of the associated code.
      + **word.list %>% map_chr(var_names.v[j])** creates a character vector containing the value of each row in the column indicated by **var_names.v[j]**. The function **map_chr( )** applies its second argument to every element of its first argument (here **word.list**, on the left of the pipe operator). The second argument is simply the name of the *jth* column as corresponds to the present iteration of the loop.
      + **%>% data.frame(check.names = FALSE, stringsAsFactors = FALSE)** converts the piped character vector to the first column of a data frame. The arguments **check.names** and **stringsAsFactors** should both be set to FALSE.
    + **else { ... }** selects all iterations subsequent to the first. These data are added as columns to the existing **base.df**. The associated code is the same as in the **if( )** branch except for the last step:
      + **cbind(base.df,  ., stringsAsFactors = FALSE)** joins its input arguments, in order, as columns in a data frame. Here the first argument is **base.df**, and the second argument is the result of the code preceding the pipe operator and is here represented by a dot.  
  + **colnames(base.df) <- var_names.v** the attribute/variable names in **var_names.v** are assigned as the column names for **base.df**.  
  + **selected_vars.list <- list()** creates an empty list object to store the results of the next loop.
  + **nomina.v <- NULL** creates a vector to store the names for the results of the next loop.
  + **for (k in 1:3) { ... }** is a loop to produce the selected number of n-ary combinations of the simplex variables contained as in **var_names.v**. The number of iterations corresponds to *k* in the binomial coefficent ${n \choose k}$. With 48 or so simplex variables to combine, the number of such possible combinations climbs rapidly. For example, ${48 \choose 2} = 1,128, {48 \choose 3} = 17,296$ and ${48 \choose 4} = 194,580$. Thus, a maximum of three simplex variables as the basis of the combined variables seems a reasonable place to start. 
    + **selected_vars.list[[k]] <- combn(var_names.v, k, simplify = FALSE)** stores in the list element corresponding to the *kth* iteration of the loop combinations of simplex variables indicated. The function **combn( )** takes the elements of the first input argument and returns all n-ary combinations, with the cardinality of the n-ary sets being determined by the second argument of **combn( )**. For example, during the second iteration of the loop, *k* will be equal to 2. All two-element combinations of the variable names in **var_names.v**. When the argument **simpify** is set to FALSE, the function returns a list. Here, the result is a list of ${n \choose k}$ elements, where n = the number of elements in **var_names.v** and k = the current iteration of the loop. Each of these list elements contains a character vector with elements equal in number to k.
    + **nomina.v <-** populates the chaacter vector with the results of the associated code.
    + **paste(length(var_names.v), "Choose",  k, collapse = " ")** creates a character vector of the type "48 Choose 2".
    + **%>% append(nomina.v, .)** pipes the output of the previous line to the **append( )** function. This function joins the piped input (represented among the function aruments as a dot) to the **nomina.v** vector. 
  + **names(selected_vars.list) <- nomina.v** assigns the elements of **nomina.v** as names to the main list elements of the list indicated.
  + **new_vars.m <- matrix(nrow = nrow(base.df), ncol = 1)** initiates a matrix object for the results of the following loop.
  + **for (m in seq_along(selected_vars.list)) { ... }** is a loop to extract values for the new variable sets (simplex and combined).
    + **a <- sapply(selected_vars.list[[m]], generate_vars, df = base.df)** creates a matrix containg a row for each token in the input data frame and a column for each character vector in the input list element.
      + **sapply( )** applies to each element of **selected_vars.list[[m]]** the user-defined function **generate_vars**.
    + **nomina <- sapply(selected_vars.list[[m]], generate_var_names)** creates a character vector of names for the variables. 
    + **colnames(a) <- nomina** applies the elements of **nomina** as names for the columns in **a**.
    + **new_vars.m <- cbind(new_vars.m, a)** adds the columns of matrix **a** to **new_vars.m**.
  + **new_vars.m <- new_vars.m[, -1]** removes the first (empty) column from the matrix.
  + **new_vars.m[str_detect(new_vars.m, "NA")] <- NA** replaces the character string "NA" with the logical value NA. This step is necessary to more easily eliminate empty variables. The function **str_detect( )** is from the stringr package.
  + **new_vars.m[str_detect(new_vars.m, "NULL")] <- NA** replaces the character string "NULL" with the logical value NA.
  + **new_vars.m <-tolower(new_vars.m)** converts all variables values and variable names to lower case character strings. This step is necessary to avoid mis-identification of variables and values.
  + **vars.tib <- as_tibble(new_vars.m)** converts the matrix to a tibble object. This step allows use of convenient functions.
  + **file_var_ct.tib <-** creates a new tibble containing a row for each variable type and two columns, one containing the names of the variable types, the other the count of the number of times that particular variable type has a significant value. This tibble is populated by the associated code:
    + **gather(vars.tib, variable_name, variable_value, na.rm = TRUE)** creates a tibble with the number of rows equal to the number of cells in the input, **vars.tib**, with significant (non-NA) values. There are two columns, one containing the variable name and the other the variable value.
    + **%>% group_by(variable_name)** pipes the results of the previous code to the **group_by( )** function and returns a tibble in which rows are grouped by contents of the column **variable_name**.
    + **%>% summarize(n())** pipes the results of the previous code to the **summarize( )** function, which summarizes multiple values to a single value. In other words, it returns a tibble of one row for each unique variable name. The function **n( )** counts the intances of each variable of a given name and returns this value in a second column.
  + **holder.tib <- bind_rows(holder.tib, file_var_ct.tib)** adds the tibble of variable names and counts to the storage tibble created before the rinning of this loop. The function **bind_rows( )** adds, row by row, the second input argument to the first.
* This is the end of the main loop.  The next step is to generate a single aggregated contingency table for the data from all xml input files.  

  


In [ ]:
# aggregation of results

agg_var_ct.tib  <- xtabs(`n()` ~ variable_name, data = holder.tib) %>%
  as_tibble() # combine all frequencies in holder.tib to in one table


agg_var_ct.tib$n %>%
  quantile(., probs = (seq(0.9, 1, 0.005)))

shorter_var.tib <- agg_var_ct.tib %>% 
  filter(n >= 52960) %>% # select value that will give reasonable number of variables
  arrange(desc(n))


keepers.v <- NULL

keepers.v <- append(keepers.v,   apply(shorter_var.tib[,1], 1, `[`))

reduced_atomic_vars.list <- sapply(keepers.v, strsplit, split = "_&_")

saveRDS(reduced_atomic_vars.list, file = "reduced_var_list.rds")


### The final step is to aggregate results from each file in a single frequency table and cull and save results.

* **agg_var_ct.tib  <-** creates tibble object to hold results of associated code. The content is a contingency table with variable names and frequencies for all xml files aggregated as a whole.
  + **xtabs(\`n()\` ~ variable_name, data = holder.tib)** creates the contingency table using the forumula given by the first argument. On the left hand of the tilde is the column with counts, on the right side, the variable(s) to cross classify against the counts. 
  + **%>% as_tibble()** converts the piped input to a tibble object.
* **agg_var_ct.tib&#36;n %>%** selects the column of the named tibble holding the frequency counts and pipes them as input to the next function.  
  + **quantile(., probs = (seq(0.9, 1, 0.005)))** returns for the frequencies input the quantiles between 90% and 100% in 0.5% increments. These numbers are used to select a reasonable number of variables to use for classification. Selecting from the most frequent 10% of variables make sense, since there will be fewer empty values.
* **shorter_var.tib <- agg_var_ct.tib** creates a tibble of the selected variables, based on the associated code.
  + **%>% filter(n >= 52960)** selects those rows of the piped input which meet the criterion specified. Here, the selection is for those rows whose n column contains a nunmber equal to or greater than the figure indicated.
  + **%>% arrange(desc(n))** pipes the results of the previous line to the **arrange( )** function, which orders the rows. The **desc(n)** function used as an argument for **arrange( )** indicates an order from highest to lowest values in column n.
* **keepers.v <- NULL** creates an empty vector for the **append( )** function.
* **keepers.v <- append(keepers.v,   apply(shorter_var.tib[,1], 1, \`[\`))** populates the vector with the name of the varialbes to be kept.
  + **apply(shorter_var.tib[,1], 1, \`[\`)** applies the **[** function to each row of the first column of the input tibble. This function returns the value of the cell in question (here the name of the variable).
* **reduced_atomic_vars.list <- sapply(keepers.v, strsplit, split = "_&_")** creates a list object containing a character vector of one or more elements for each variable name in **keepers.v**. Each element is the name of a simplex variable from which combined variables may be created.
* **saveRDS(reduced_atomic_vars.list, file = "reduced_var_list.rds")** saves the results to disk for later use.
  
